In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install transformers
!pip install datasets
!pip install bitsandbytes
!pip install peft
!pip install trl
!pip install accelerate
!pip install llmcompressor

In [ ]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from llmcompressor.transformers import SparseAutoModelForCausalLM, oneshot
from llmcompressor.modifiers.quantization import GPTQModifier

In [ ]:
from datasets import load_dataset
dataset = load_dataset("llama-duo/gemma2b-coding-eval-by-claude3sonnet")

In [ ]:
dataset['gemma2b_coding_gpt4o_100k_by_claude3sonnet'][0]

In [ ]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['instructions'])):
        prompt_list.append(r"""<bos><start_of_turn>user
다음 명령에 따라 코드를 작성해 주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['instructions'][i], example['target_responses'][i]))
    return prompt_list

In [ ]:
train_data = dataset['gemma2b_coding_gpt4o_100k_by_claude3sonnet']

In [ ]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
BASE_MODEL = "google/gemma-2-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
#        num_train_epochs = 1,
        max_steps=300,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        optim="paged_adamw_8bit",
        warmup_steps=90,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)

In [ ]:
!ls -alh lora_adapter

In [ ]:
!pip install peft

In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel # Import PeftModel from peft

#from huggingface_hub import login

# Hugging Face에 로그인
#login(token="your_huggingface_token")

# 모델을 불러오고 준비
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

# 모델 병합 및 언로드
model = model.merge_and_unload()

# 로컬에 모델 저장
model.save_pretrained('gemma-2-2b-it-coding')

# 모델을 허브에 푸시
model.push_to_hub("gemma-2-2b-it-coding", commit_message="Initial commit of merged model for IT coding tasks.")